<a href="https://colab.research.google.com/github/Sar-thak-3/Extract/blob/main/TextToQuestions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
f1 = open("/content/drive/MyDrive/TextQuestions/train_v0.2.json")
data1 = json.load(f1)
f2 = open("/content/drive/MyDrive/TextQuestions/train-v1.1.json")
data2 = json.load(f2)

In [ ]:
# questions = []
# paragraphs = []

# for item in data1["data"][:]:
#   paraques = []
#   para = []
#   for context in item["paragraphs"]:
#     para.append(context["context"])
#     contextques = []
#     for qas in context["qas"]:
#       contextques.append(qas["question"])

#     paraques.append(contextques)

#   questions.append(paraques)
#   paragraphs.append(para)

# for item in data2["data"][:]:
#   paraques = []
#   para = []
#   for context in item["paragraphs"]:
#     para.append(context["context"])
#     contextques = []
#     for qas in context["qas"]:
#       contextques.append(qas["question"])

#     paraques.append(contextques)

#   questions.append(paraques)
#   paragraphs.append(para)

In [ ]:
questions = []
paragraphs = []

for item in data1["data"][:]:
  for context in item["paragraphs"]:
    para = context["context"]
    ques = ""
    for qas in context["qas"]:
      ques += qas["question"]

    questions.append(ques)
    paragraphs.append(para)


for item in data2["data"][:]:
  for context in item["paragraphs"]:
    para = context["context"]
    ques = ""
    for qas in context["qas"]:
      ques += qas["question"]
      ques += " "
      # questions.append(qas["question"])

    questions.append(ques)
    paragraphs.append(para)

In [ ]:
questions[1]

'what language do they speak?Do they speak any other languages?any literary items of interest?How old is their literature?were any of the poets listed by name?anything else of interest?any more recent literary works from them?'

In [ ]:
len(questions)

30463

In [ ]:
len(paragraphs)

30463

In [ ]:
# Regexp of text and questions

In [ ]:
import re
def clean_text(sentence):
  sentence = sentence.lower()
  sentence = re.sub("[^a-z]+"," ",sentence)
  sentence = sentence.split()
  sentence = " ".join(sentence)
  return sentence

In [ ]:
clean_text(questions[0])

'where is malayali located what other languages are spoken there what else is this place known for were they ever successful in doing this do they produce anything from here is this population still growing is the country thriving'

In [ ]:
for index,ques in enumerate(questions):
  questions[index] = clean_text(ques)

In [ ]:
for index,para in enumerate(paragraphs):
  paragraphs[index] = clean_text(para)

In [ ]:
import numpy as np

f = open("/content/drive/MyDrive/Image_captioning/glove.6B.50d.txt" , encoding='utf8')
embedding_index = {}
embedding_to_word = {}

for line in f:
  values = line.split()
  word = values[0]
  word_embedding = np.array(values[1:],dtype='float')
  embedding_to_word[word_embedding] = word
  embedding_index[word] = word_embedding

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = stopwords.words("english")
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
wordnet_lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
clean_questions = []
clean_paragraphs = []

for ques in questions:
  qs = []
  [qs.append(wordnet_lemmatizer.lemmatize(word)) for word in ques.split() if word in embedding_index.keys() and word not in stopwords]
  clean_questions.append(" ".join(qs))

In [ ]:
for para in paragraphs:
  ps = []
  [ps.append(wordnet_lemmatizer.lemmatize(word)) for word in para.split() if word in embedding_index.keys() and word not in stopwords]
  clean_paragraphs.append(" ".join(ps))

In [ ]:
total_words = []
for ques in clean_questions:
    [total_words.append(word) for word in ques.split()]
print(len(total_words))

667975


In [ ]:
for para in clean_paragraphs:
  [total_words.append(word) for word in para.split()]
print(len(total_words))

4523823


In [ ]:
import collections
counter = collections.Counter(total_words)
freq_cnt = dict(counter)

In [ ]:
vocab = set()
for ques in clean_questions:
  [vocab.update(ques.split())]
print(len(vocab))

for para in clean_paragraphs:
  [vocab.update(para.split())]
print(len(vocab))

31085
83373


In [ ]:
word_to_idx = {}
idx_to_word = {}
count = 0
for i,word in enumerate(vocab):
  if(freq_cnt[word]>3):
    word_to_idx[word] = count
    idx_to_word[count] = word
    count+=1

In [ ]:
len(word_to_idx)

44306

In [ ]:
length = len(word_to_idx)

In [ ]:
length

44306

In [ ]:
word_to_idx['startseq'] = length+1
word_to_idx['endseq'] = length+2
idx_to_word[length+1] = 'startseq'
idx_to_word[length+2] = 'endseq'

In [ ]:
len(word_to_idx)

44308

In [ ]:
len(clean_questions)

30463

In [ ]:
# Adding startseq and endseq to every questions
for index,ques in enumerate(clean_questions):
  clean_questions[index] = "startseq " + ques + " endseq"

In [ ]:
clean_questions[0]

'startseq malayali located language spoken else place known ever successful produce anything population still growing country thriving endseq'

In [ ]:
context_maxlen = 0
for cont in clean_paragraphs:
  context_maxlen = max(context_maxlen,len(cont.split()))
question_maxlen = 0
for ques in clean_questions:
  question_maxlen = max(question_maxlen,len(ques.split()))

In [ ]:
context_maxlen

942

In [ ]:
question_maxlen

128

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense,Dropout,Embedding,LSTM,Input
from keras.layers.merging import add
from keras.models import Model
from keras.utils import to_categorical

In [ ]:
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.TPUStrategy(tpu)

In [ ]:
vocab_size = len(word_to_idx)+1

In [ ]:
def datagenerator(vocab_size=vocab_size,context_data=clean_paragraphs,questions_data=clean_questions,maxlen_context=context_maxlen,maxlen_questions=question_maxlen,word_to_idx=word_to_idx,batch_size=14):
  context = []
  questions = []
  generated = []
  n=0

  while(True):
    for index in range(len(context_data)):
      n+=1
      para_sequence = [word_to_idx[word] for word in context_data[index].split() if word in word_to_idx.keys()]
      xi2 = pad_sequences([para_sequence],maxlen=maxlen_context,value=0,padding='post')[0]
      ques_sequence = [word_to_idx[word] for word in questions_data[index].split() if word in word_to_idx.keys()]
      for i in range(1,len(ques_sequence)):
        xi = ques_sequence[0:i]
        yi = ques_sequence[i]
        yi = to_categorical([yi-1],num_classes=vocab_size)[0]
        xi = pad_sequences([xi],maxlen=maxlen_questions,value=0,padding='post')[0]

        context.append(xi2)
        questions.append(xi)
        generated.append(yi)

      if(n==batch_size):
        yield [[tf.convert_to_tensor(context),tf.convert_to_tensor(questions)],tf.convert_to_tensor(generated)]
        # yield [[np.array(context),np.array(questions)],np.array(generated)]
        context,questions,generated = [],[],[]
        n = 0

In [ ]:
def get_embedding_matrix(vocab_size=len(word_to_idx)):
  emb_dim = 50
  matrix = np.zeros((vocab_size+1,emb_dim))
  for word,idx in word_to_idx.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
      matrix[idx] = embedding_vector
  return matrix

In [ ]:
embedding_matrix = get_embedding_matrix()

In [ ]:
with tf.device('/device:GPU:0'):
  embedding_matrix = tf.convert_to_tensor(embedding_matrix)

In [ ]:
embedding_matrix.shape

TensorShape([44309, 50])

In [ ]:
input_context = Input(shape=(context_maxlen,))
input_context1 = Embedding(input_dim=vocab_size,output_dim=50,mask_zero=True)(input_context)
input_context2 = LSTM(128)(input_context1)
# input_context3 = Dense(256,activation="relu")(input_context2)

input_questions = Input(shape=(question_maxlen,))
input_questions1 = Embedding(input_dim=vocab_size,output_dim=50,mask_zero=True)(input_questions)
input_questions2 = LSTM(128)(input_questions1)
# input_questions3 = Dense(256,activation="relu")(input_questions2)

decoder = add([input_context2,input_questions2])
decoder1 = Dense(512,activation="relu")(decoder)
outputs = Dense(vocab_size,activation="softmax")(decoder1)

model = Model(inputs=[input_context,input_questions],outputs=outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 942)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 942, 50)      2215450     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 128, 50)      2215450     ['input_2[0][0]']                
                                                                                              

In [ ]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False
model.layers[3].set_weights([embedding_matrix])
model.layers[3].trainable = False

with tf.device('/device:GPU:0'):
  model.compile(loss="categorical_crossentropy",optimizer="adam")
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 942)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 942, 50)      2215450     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 128, 50)      2215450     ['input_2[0][0]']                
                                                                                              

In [ ]:
with tf.device('/device:GPU:0'):
  epochs = 5
  batch_size = 14
  steps = len(clean_questions)//batch_size
  for i in range(epochs):
    generator = datagenerator(vocab_size,clean_paragraphs,clean_questions,context_maxlen,question_maxlen,word_to_idx,batch_size)
    model.fit(generator,epochs=1,steps_per_epoch=steps,verbose=1,shuffle=True)

  48/2175 [..............................] - ETA: 47:39 - loss: 6.2091

# For TPU

In [ ]:
with tpu_strategy.scope():
  vocab_size = len(word_to_idx)+1
  input_context = Input(shape=(context_maxlen,))
  input_context1 = Embedding(input_dim=vocab_size,output_dim=50,mask_zero=True)(input_context)
  input_context2 = LSTM(128)(input_context1)

  input_questions = Input(shape=(question_maxlen,))
  input_questions1 = Embedding(input_dim=vocab_size,output_dim=50,mask_zero=True)(input_questions)
  input_questions2 = LSTM(128)(input_questions1)

  decoder = add([input_context2,input_questions2])
  decoder1 = Dense(512,activation="relu")(decoder)
  outputs = Dense(vocab_size,activation="softmax")(decoder1)

  model = Model(inputs=[input_context,input_questions],outputs=outputs)

  model.layers[2].set_weights([embedding_matrix])
  model.layers[2].trainable = False
  model.layers[3].set_weights([embedding_matrix])
  model.layers[3].trainable = False

  model.compile(loss="categorical_crossentropy",optimizer="adam")
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 942)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 942, 50)      2215450     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 128, 50)      2215450     ['input_2[0][0]']                
                                                                                              

In [ ]:
epochs = 5
batch_size = 14
steps = len(clean_questions)//batch_size
for i in range(epochs):
  dataset = tf.data.Dataset.from_generator(datagenerator,output_types=(tf.dtypes.float32),output_shapes=(1))
  print(dataset)
  # generator = datagenerator(vocab_size,clean_paragraphs,clean_questions,context_maxlen,question_maxlen,word_to_idx,batch_size)
  model.fit(dataset,epochs=1,steps_per_epoch=steps,verbose=1,shuffle=True)

<_FlatMapDataset element_spec=TensorSpec(shape=(1,), dtype=tf.float32, name=None)>


ValueError: ignored

In [ ]:
# for data in range(0,1):
generator = datagenerator(vocab_size,clean_paragraphs,clean_questions,context_maxlen,question_maxlen,word_to_idx,10)
print(model.predict(generator))

    197/Unknown - 162s 826ms/step

In [ ]:
epochs = 10
batch_size = 14
steps = len(clean_questions)//batch_size
for i in range(epochs):
  generator = datagenerator(vocab_size,clean_paragraphs,clean_questions,context_maxlen,question_maxlen,word_to_idx,batch_size)
  model.fit(generator,epochs=1,steps_per_epoch=steps,verbose=1,shuffle=True)
  break

tf.Tensor(
[[33725  1583  4466 ...     0     0     0]
 [33725  1583  4466 ...     0     0     0]
 [33725  1583  4466 ...     0     0     0]
 ...
 [27094 36226 31283 ...     0     0     0]
 [27094 36226 31283 ...     0     0     0]
 [27094 36226 31283 ...     0     0     0]], shape=(287, 942), dtype=int32)
tf.Tensor(
[[27094 27815 11249 ...     0     0     0]
 [27094 27815 11249 ...     0     0     0]
 [27094 27815 11249 ...     0     0     0]
 ...
 [30542   624 15448 ...     0     0     0]
 [30542   624 15448 ...     0     0     0]
 [30542   624 15448 ...     0     0     0]], shape=(266, 942), dtype=int32)


KeyboardInterrupt: ignored